#### Imports

In [114]:
import numpy
import os
import pandas as panda

import matplotlib.pyplot as plot
from matplotlib.ticker import FuncFormatter

import calendar 
from datetime import datetime
from collections import defaultdict, Counter

from odf.opendocument import load
from odf.table import Table


#### Days in the current month

In [ ]:
# Get the current year and month and time
current_date = datetime.now()
current_year = current_date.year
current_month = current_date.month

# Extract the current month's name
current_month_name = current_date.strftime("%B")

# Get the number of days in the current month
_, daysInCurrentMonth = calendar.monthrange(current_year, current_month)

print(f"The current month is {current_month_name} and it has {daysInCurrentMonth} days.")


#### Data handling

In [ ]:
ods_file_path = os.path.expanduser('~/Downloads/data.ods')

if not os.path.isfile(ods_file_path):
    print("File not found: ", ods_file_path)

ods_file = load(ods_file_path)

# Get all tables
sheets = ods_file.getElementsByType(Table)
num_sheets = len(sheets)

if num_sheets == 0:
    print("No sheets found in the document.")
    exit(0)
    
sheet_name = sheets[0].getAttribute("name")

# read the ods file
ods_data = panda.read_excel(ods_file_path, sheet_name=sheet_name, engine='odf')

# extract the data from the columns in the ods file and convert them into a list
expenses = ods_data['value'].dropna().abs().values.tolist()
dates = ods_data['date'].dropna().values.tolist()

# day-month-year instead of day.month.year
dates = [ date.replace('.', '-') for date in dates ]

# convert them into datetime objects and inverse
dates = [ datetime.strptime(date, '%d-%m-%Y').date().strftime('%d-%m-%y') for date in dates ]

# create a dictionary with the pair (key, value) over tuple(key : value)
date_expense_dict = { (date, expense) for date, expense in zip(dates, expenses) }

# takes a list and returns one which is sorted, rounded and without any redundancy
def aggregate_expenses(dates_expenses):
    # Initialize a default dictionary to store the aggregated expenses
    expenses_dict = defaultdict(float)
    
    # Iterate through the list of date-expense pairs
    for date, expense in dates_expenses:
        # Aggregate the expenses for each date
        expenses_dict[date] += expense
    
    # round the expenses
    rounded_expenses = { 
        date : round(expense, 4) for date, expense in expenses_dict.items() 
    }
    
    # 'key' is executed before any sorting takes place; take x which is a tuple (key, value), get the date with x[0] and convert it into a datetime object
    sorted_expenses = dict(sorted(rounded_expenses.items(), key=lambda x: datetime.strptime(x[0], "%d-%m-%y")))
    
    return sorted_expenses

date_expense_dict = aggregate_expenses(date_expense_dict)

print(date_expense_dict)


In [241]:
#
get_month = lambda x : datetime.strptime(x, "%d-%m-%y").date().month

#
months_in_dict = [ str(get_month(date)) for date in date_expense_dict ]

#
months_in_dict_as_list = list(Counter(months))

In [242]:
#
def group_expenses_by_month(dictionary):
    #
    new_dict = defaultdict(float)
    
    #
    for date, expense in dictionary.items():
        
        month = '0' + str(get_month(date))
        
        if month in months_in_dict_as_list:
            new_dict[date[3:]] += float(f'{expense:.2f}')
    
    return dict(new_dict)

date_expense_dict = group_expenses_by_month(date_expense_dict)


#### Dataframe

In [ ]:
# creates the dataframe with the give columns
dataframe = panda.DataFrame(list(date_expense_dict.items()), columns=['date', 'expenses'])

dataframe

In [234]:

# calculates the average
average = numpy.mean(dataframe['expenses'])


#### Graph
Displays the graph with the given data and the current month

In [ ]:
# Changes the size of the bars inside the plot
plot.figure(figsize=(14, 8))

# data of the x-axis/bar value is assigned as they need to be identical
xdata = dataframe['expenses']

# stores the bars inside the variable: how many, the actual values
bars = plot.bar(range(len(dataframe['expenses'])), xdata)

# creates the bars inside the diagram
plot.bar(dataframe['date'], xdata, color='skyblue')

# x: value of tick on an axis; pos: where the tick is located
def currency_formatter(x, pos):
    return f'{x:.0f}€'

# get current axes (gca)
plot.gca().yaxis.set_major_formatter(FuncFormatter(currency_formatter))

for bar in bars:
    y_axis_value = bar.get_height() # get the y-axis height which corresponds with the y value
    # adds text annotation to the bar; left edge + center = center, y, value 
    plot.text(bar.get_x() + bar.get_width()/2, y_axis_value, f'{y_axis_value:.2f}€', ha='center', va='bottom')

# change the x and y-bar label font size
plot.tick_params(axis='both', labelsize=11)

# axis Labels, font size and distance to the x bar labels
plot.xlabel('Datum', fontsize=18, labelpad=16)
plot.ylabel('Ausgaben', fontsize=18, labelpad=16)

# Title
plot.title(f'Ausgaben {current_month_name}', fontsize=20)

# Rotate the x bar labels by some degree
plot.xticks(rotation=45)

#y: where to line will be drawn
plot.axhline(y=average, color='red', linestyle='-', label=f'Average: {average:.2f}€')

# 
plot.tight_layout()

# displays a legend
plot.legend()

# Shows the plot
plot.show()


In [ ]:
sum = dataframe['expenses'].sum()

print(f'Summe aller monatlichen Ausgaben: {sum:.2f}')

## Alternative Approaches

##### 1. create a boolean mask whether a value is nan

boolean_mask = ~numpy.isnan(expenses)
#####
##### 2. remove any value that is nan

expenses = [ exp for exp, valid in zip(expenses, boolean_mask) if valid ]
dates = [ date for date, valid in zip(dates, boolean_mask) if valid ]
'''


#### Get all the days in a month exercise

In [246]:
# Initializations
i: int = 1
dates = []

# Create an array for the days of the month
while i <= daysInCurrentMonth:
    dates.append(f'{i}-{current_date.strftime("%m-%Y")}')
    i += 1
    

#### User Input Exercise

In [ ]:
'''
import sys

from IPython.core.display_functions import clear_output

userinput: str = ''
spendingdata: list[float] = []


#
def getspendings() -> str:
    global spendingdata
    global userinput

    # Get the user input
    userinput = input('Gebe deine Ausgaben ein: ')

    if userinput == 'end':
        return userinput
    else:
        # try to int conversion
        try:
            userinput = float(userinput)
        except ValueError:
            clear_output(wait=True)
            print('Please make sure the input is a number!')
            sys.exit(1)

    # validate the input
    if type(userinput) is float:
        
        if daysInCurrentMonth <= len(spendingdata):
            print('Done!')
            return 'end'
        
        spendingdata.append(userinput)
        print(f'Appended {userinput}')

        return userinput
    else:
        clear_output(wait=True)
        print('Something went wrong!')
        sys.exit(1)


while userinput != 'end':
    userinput = getspendings()
'''